<a href="https://colab.research.google.com/github/Justus-coded/Competitions/blob/master/DSN%20AI%20Bootcamp%20Qualification%20Hackathon/DSN%20AI%20Boot_camp%20Qualification1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#train = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Train.csv')
#test  = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Test.csv')

train = pd.read_csv('../input/dsn-ai-bootcamp-data/Train.csv')
test = pd.read_csv('../input/dsn-ai-bootcamp-data/Test.csv')
sub = pd.read_csv('../input/dsn-ai-bootcamp-data/SampleSubmission.csv')

In [ ]:
dict_ = {'charge': 1,
         'lending': 0}
 
target_map = {
    'yes': 1,
    'no': 0}
 
train.default_status.replace(target_map, inplace = True)
 
train['form_field47'].replace(dict_, inplace = True)
test['form_field47'].replace(dict_, inplace = True)

In [ ]:
#pip install catboost

In [ ]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [ ]:
features = train.select_dtypes(exclude = object).columns.drop(['default_status'])

In [ ]:
# features

In [ ]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

In [ ]:
train['form_field1*formfield2'] = train['form_field1'] * train['form_field2']
train['form_field1/formfield2'] = train['form_field1'] / train['form_field2']

train['form_field1*formfield3'] = train['form_field1'] * train.form_field3
train['form_field1*formfield4'] = train.form_field1 * train.form_field4
train['form_field1*formfield5'] = train.form_field1 * train.form_field5

#new_train['form_field30*formfield41'] = new_train['form_field30'] * new_train['form_field41']
##new_train['form_field31*formfield40'] = new_train['form_field31'] * new_train['form_field40']

train['form_field6+field7'] =  (train.form_field6 + train.form_field8)/2

train['form_field1*formfield6'] = train['form_field1'] * train['form_field6']
train['form_field2*formfield6'] = train['form_field2'] * train['form_field6']

train['form_field1*formfield9'] = train['form_field1'] * train['form_field9']
train['form_field2*formfield9'] = train['form_field2'] * train['form_field9']

train['form_field1*formfield37'] = train['form_field1'] * train['form_field37']
train['form_field2*formfield37'] = train['form_field2'] * train['form_field37']

train['form_field9*formfield6'] = train['form_field9'] * train['form_field6']
train['form_field37*formfield6'] = train['form_field37'] * train['form_field6']

train['form_field9*formfield37'] = train['form_field9'] * train['form_field37']
train['form_field1*formfield8'] = train['form_field1'] * train['form_field8']

In [ ]:
test['form_field1*formfield2'] = test['form_field1'] * test['form_field2']
test['form_field1/formfield2'] = test['form_field1'] / test['form_field2']

test['form_field1*formfield3'] = test['form_field1'] * test.form_field3
test['form_field1*formfield4'] = test.form_field1 * test.form_field4
test['form_field1*formfield5'] = test.form_field1 * test.form_field5

test['form_field6+field7'] =  (test.form_field6 + test.form_field8)/2

test['form_field1*formfield6'] = test['form_field1'] * test['form_field6']
test['form_field2*formfield6'] = test['form_field2'] * test['form_field6']

test['form_field1*formfield9'] = test['form_field1'] * test['form_field9']
test['form_field2*formfield9'] = test['form_field2'] * test['form_field9']

test['form_field1*formfield37'] = test['form_field1'] * test['form_field37']
test['form_field2*formfield37'] = test['form_field2'] * test['form_field37']

test['form_field9*formfield6'] = test['form_field9'] * test['form_field6']
test['form_field37*formfield6'] = test['form_field37'] *test['form_field6']

test['form_field9*formfield37'] = test['form_field9'] * test['form_field37']
test['form_field1*formfield8'] = test['form_field1'] * test['form_field8']

In [ ]:
X = train[features]
y = train['default_status']

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [ ]:
# Specify number of folds
n_skf = 15
kf = StratifiedKFold(n_skf)
 
#seed = 42
 
## Manually tweak the hyperparameter, learning rate and n_estimators especially
params = {'max_depth':7,
    'n_estimators': 10000,
    'learning_rate': 0.005,
    'objective': 'CrossEntropy',
      'eval_metric':'AUC',
    
    'early_stopping_rounds': 200,
    'use_best_model': True,
}

In [ ]:
score_list = []
score = 0
test_oofs = []
 
for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]
 
    model = CatBoostClassifier(**params, task_type='GPU',random_seed = i)
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p = model.predict_proba(xval)[:, 1]
    sc = metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)
 
    print('Fold {} : {}'.format(i, sc))
 
print()
print()
print('Avg log : ', score)

0:	learn: 0.8062807	test: 0.7964385	best: 0.7964385 (0)	total: 17.3ms	remaining: 2m 52s
100:	learn: 0.8279872	test: 0.8135664	best: 0.8135920 (99)	total: 1.57s	remaining: 2m 33s
200:	learn: 0.8325671	test: 0.8168218	best: 0.8168218 (200)	total: 3.09s	remaining: 2m 30s
300:	learn: 0.8361511	test: 0.8192585	best: 0.8192585 (300)	total: 4.67s	remaining: 2m 30s
400:	learn: 0.8390048	test: 0.8208697	best: 0.8208697 (400)	total: 6.54s	remaining: 2m 36s
500:	learn: 0.8411735	test: 0.8219254	best: 0.8219254 (500)	total: 8.42s	remaining: 2m 39s
600:	learn: 0.8429645	test: 0.8228748	best: 0.8228775 (599)	total: 9.96s	remaining: 2m 35s
700:	learn: 0.8447050	test: 0.8236692	best: 0.8236692 (700)	total: 11.5s	remaining: 2m 32s
800:	learn: 0.8462695	test: 0.8242500	best: 0.8242511 (797)	total: 13s	remaining: 2m 28s
900:	learn: 0.8477381	test: 0.8249028	best: 0.8249118 (893)	total: 14.5s	remaining: 2m 26s
1000:	learn: 0.8491815	test: 0.8255677	best: 0.8255786 (997)	total: 16s	remaining: 2m 23s
1100:	

1700:	learn: 0.8559534	test: 0.8461680	best: 0.8461680 (1700)	total: 27.2s	remaining: 2m 12s
1800:	learn: 0.8570026	test: 0.8464316	best: 0.8464316 (1800)	total: 28.7s	remaining: 2m 10s
1900:	learn: 0.8580021	test: 0.8465774	best: 0.8465917 (1896)	total: 30.2s	remaining: 2m 8s
2000:	learn: 0.8589674	test: 0.8467243	best: 0.8467243 (2000)	total: 32.2s	remaining: 2m 8s
2100:	learn: 0.8600033	test: 0.8469457	best: 0.8469577 (2098)	total: 33.8s	remaining: 2m 7s
2200:	learn: 0.8610989	test: 0.8471007	best: 0.8471007 (2200)	total: 35.3s	remaining: 2m 5s
2300:	learn: 0.8621290	test: 0.8472589	best: 0.8472589 (2300)	total: 36.9s	remaining: 2m 3s
2400:	learn: 0.8631778	test: 0.8474439	best: 0.8474462 (2399)	total: 38.4s	remaining: 2m 1s
2500:	learn: 0.8642160	test: 0.8476125	best: 0.8476179 (2495)	total: 40s	remaining: 1m 59s
2600:	learn: 0.8652967	test: 0.8478188	best: 0.8478188 (2600)	total: 41.7s	remaining: 1m 58s
2700:	learn: 0.8663589	test: 0.8479583	best: 0.8479642 (2697)	total: 43.5s	rem

500:	learn: 0.8397340	test: 0.8416871	best: 0.8416871 (500)	total: 9.23s	remaining: 2m 55s
600:	learn: 0.8416826	test: 0.8428127	best: 0.8428127 (600)	total: 11.4s	remaining: 2m 57s
700:	learn: 0.8434278	test: 0.8438250	best: 0.8438250 (700)	total: 13.3s	remaining: 2m 56s
800:	learn: 0.8450343	test: 0.8443098	best: 0.8443098 (800)	total: 14.9s	remaining: 2m 50s
900:	learn: 0.8464816	test: 0.8448814	best: 0.8448814 (900)	total: 16.4s	remaining: 2m 45s
1000:	learn: 0.8478860	test: 0.8454161	best: 0.8454161 (1000)	total: 18s	remaining: 2m 41s
1100:	learn: 0.8492255	test: 0.8459162	best: 0.8459162 (1100)	total: 19.5s	remaining: 2m 37s
1200:	learn: 0.8504438	test: 0.8462856	best: 0.8462856 (1200)	total: 21s	remaining: 2m 33s
1300:	learn: 0.8516111	test: 0.8465272	best: 0.8465307 (1297)	total: 22.9s	remaining: 2m 32s
1400:	learn: 0.8527161	test: 0.8467272	best: 0.8467412 (1395)	total: 24.5s	remaining: 2m 30s
1500:	learn: 0.8538927	test: 0.8470102	best: 0.8470129 (1499)	total: 26s	remaining: 

900:	learn: 0.8470617	test: 0.8383585	best: 0.8383585 (900)	total: 14s	remaining: 2m 21s
1000:	learn: 0.8483849	test: 0.8388829	best: 0.8388829 (1000)	total: 15.9s	remaining: 2m 23s
1100:	learn: 0.8497496	test: 0.8393323	best: 0.8393323 (1100)	total: 17.4s	remaining: 2m 20s
1200:	learn: 0.8510323	test: 0.8398361	best: 0.8398430 (1195)	total: 19s	remaining: 2m 19s
1300:	learn: 0.8522576	test: 0.8402397	best: 0.8402424 (1295)	total: 20.5s	remaining: 2m 17s
1400:	learn: 0.8533778	test: 0.8404861	best: 0.8404989 (1393)	total: 22s	remaining: 2m 15s
1500:	learn: 0.8544375	test: 0.8408013	best: 0.8408021 (1499)	total: 23.5s	remaining: 2m 13s
1600:	learn: 0.8555389	test: 0.8410050	best: 0.8410062 (1599)	total: 25.1s	remaining: 2m 11s
1700:	learn: 0.8565444	test: 0.8412034	best: 0.8412034 (1700)	total: 27.1s	remaining: 2m 12s
1800:	learn: 0.8576092	test: 0.8414328	best: 0.8414328 (1800)	total: 28.9s	remaining: 2m 11s
1900:	learn: 0.8585832	test: 0.8417134	best: 0.8417134 (1900)	total: 30.3s	rem

3600:	learn: 0.8753359	test: 0.8436997	best: 0.8437122 (3594)	total: 59.7s	remaining: 1m 46s
3700:	learn: 0.8762186	test: 0.8437680	best: 0.8437789 (3694)	total: 1m 1s	remaining: 1m 44s
3800:	learn: 0.8771152	test: 0.8437816	best: 0.8437855 (3782)	total: 1m 2s	remaining: 1m 42s
3900:	learn: 0.8780215	test: 0.8437599	best: 0.8438123 (3840)	total: 1m 5s	remaining: 1m 41s
4000:	learn: 0.8788783	test: 0.8438379	best: 0.8438460 (3999)	total: 1m 6s	remaining: 1m 39s
4100:	learn: 0.8797486	test: 0.8439364	best: 0.8439419 (4086)	total: 1m 9s	remaining: 1m 39s
4200:	learn: 0.8806058	test: 0.8440281	best: 0.8440366 (4189)	total: 1m 10s	remaining: 1m 37s
4300:	learn: 0.8814796	test: 0.8440762	best: 0.8440995 (4238)	total: 1m 12s	remaining: 1m 35s
4400:	learn: 0.8823233	test: 0.8441371	best: 0.8441472 (4398)	total: 1m 14s	remaining: 1m 34s
4500:	learn: 0.8831562	test: 0.8441565	best: 0.8441662 (4422)	total: 1m 15s	remaining: 1m 32s
4600:	learn: 0.8839649	test: 0.8442466	best: 0.8442555 (4598)	tota

3100:	learn: 0.8707830	test: 0.8428820	best: 0.8428940 (3096)	total: 51.1s	remaining: 1m 53s
3200:	learn: 0.8717625	test: 0.8429309	best: 0.8429406 (3199)	total: 52.7s	remaining: 1m 51s
3300:	learn: 0.8727058	test: 0.8429842	best: 0.8429842 (3300)	total: 54.2s	remaining: 1m 50s
3400:	learn: 0.8736779	test: 0.8430128	best: 0.8430263 (3319)	total: 55.8s	remaining: 1m 48s
3500:	learn: 0.8746284	test: 0.8430407	best: 0.8430628 (3452)	total: 58.1s	remaining: 1m 47s
3600:	learn: 0.8755893	test: 0.8431261	best: 0.8431469 (3584)	total: 59.6s	remaining: 1m 45s
3700:	learn: 0.8765257	test: 0.8431734	best: 0.8431956 (3676)	total: 1m 1s	remaining: 1m 44s
3800:	learn: 0.8774204	test: 0.8432868	best: 0.8432895 (3799)	total: 1m 2s	remaining: 1m 42s
3900:	learn: 0.8783484	test: 0.8433594	best: 0.8433601 (3893)	total: 1m 4s	remaining: 1m 40s
4000:	learn: 0.8792490	test: 0.8433768	best: 0.8433810 (3999)	total: 1m 5s	remaining: 1m 38s
4100:	learn: 0.8801370	test: 0.8434843	best: 0.8434843 (4100)	total: 1

2200:	learn: 0.8628280	test: 0.8207366	best: 0.8207847 (2192)	total: 35.4s	remaining: 2m 5s
2300:	learn: 0.8639489	test: 0.8209081	best: 0.8209299 (2296)	total: 36.9s	remaining: 2m 3s
2400:	learn: 0.8649972	test: 0.8210933	best: 0.8210983 (2398)	total: 38.4s	remaining: 2m 1s
2500:	learn: 0.8661095	test: 0.8213479	best: 0.8213583 (2496)	total: 40.5s	remaining: 2m 1s
2600:	learn: 0.8671790	test: 0.8215292	best: 0.8215505 (2597)	total: 42s	remaining: 1m 59s
2700:	learn: 0.8682258	test: 0.8216681	best: 0.8216681 (2700)	total: 43.5s	remaining: 1m 57s
2800:	learn: 0.8692187	test: 0.8217177	best: 0.8217220 (2783)	total: 45s	remaining: 1m 55s
2900:	learn: 0.8702520	test: 0.8218377	best: 0.8218532 (2891)	total: 46.5s	remaining: 1m 53s
3000:	learn: 0.8712532	test: 0.8219223	best: 0.8219351 (2978)	total: 48s	remaining: 1m 51s
3100:	learn: 0.8722615	test: 0.8220733	best: 0.8220733 (3100)	total: 50s	remaining: 1m 51s
3200:	learn: 0.8732005	test: 0.8221311	best: 0.8221419 (3157)	total: 52.1s	remaini

2200:	learn: 0.8602513	test: 0.8665111	best: 0.8665184 (2182)	total: 35.8s	remaining: 2m 6s
2300:	learn: 0.8612946	test: 0.8664936	best: 0.8665363 (2205)	total: 37.4s	remaining: 2m 5s
2400:	learn: 0.8623310	test: 0.8664975	best: 0.8665363 (2205)	total: 39.4s	remaining: 2m 4s
bestTest = 0.8665363193
bestIteration = 2205
Shrink model to first 2206 iterations.
Fold 12 : 0.8665359241719404
0:	learn: 0.8006155	test: 0.7993405	best: 0.7993405 (0)	total: 16ms	remaining: 2m 39s
100:	learn: 0.8275405	test: 0.8161318	best: 0.8169030 (11)	total: 1.53s	remaining: 2m 29s
200:	learn: 0.8325935	test: 0.8208344	best: 0.8208344 (200)	total: 3.03s	remaining: 2m 27s
300:	learn: 0.8360962	test: 0.8238349	best: 0.8238349 (300)	total: 4.58s	remaining: 2m 27s
400:	learn: 0.8388248	test: 0.8260584	best: 0.8260584 (400)	total: 6.07s	remaining: 2m 25s
500:	learn: 0.8409193	test: 0.8277250	best: 0.8277250 (500)	total: 7.58s	remaining: 2m 23s
600:	learn: 0.8428897	test: 0.8290508	best: 0.8290508 (600)	total: 9.08

3200:	learn: 0.8713270	test: 0.8494636	best: 0.8494803 (3178)	total: 55.2s	remaining: 1m 57s
3300:	learn: 0.8722604	test: 0.8494946	best: 0.8495001 (3265)	total: 57.3s	remaining: 1m 56s
3400:	learn: 0.8732148	test: 0.8496161	best: 0.8496192 (3396)	total: 58.8s	remaining: 1m 54s
3500:	learn: 0.8741822	test: 0.8496503	best: 0.8496934 (3463)	total: 1m	remaining: 1m 52s
3600:	learn: 0.8751329	test: 0.8497407	best: 0.8497407 (3600)	total: 1m 1s	remaining: 1m 50s
3700:	learn: 0.8760591	test: 0.8497620	best: 0.8497904 (3661)	total: 1m 3s	remaining: 1m 48s
3800:	learn: 0.8770078	test: 0.8498051	best: 0.8498129 (3799)	total: 1m 5s	remaining: 1m 46s
3900:	learn: 0.8779264	test: 0.8498929	best: 0.8498929 (3900)	total: 1m 6s	remaining: 1m 44s
4000:	learn: 0.8788235	test: 0.8498703	best: 0.8498991 (3909)	total: 1m 8s	remaining: 1m 43s
4100:	learn: 0.8797185	test: 0.8499002	best: 0.8499134 (4094)	total: 1m 10s	remaining: 1m 41s
4200:	learn: 0.8805850	test: 0.8498994	best: 0.8499150 (4138)	total: 1m 

In [ ]:
f"{n_skf} fold CV, score: {score}" #Increase in local cv score should probably lead to increase in Leaderboard score

'15 fold CV, score: 0.8416762360251805'

'15 fold CV, score: 0.8416762360251805', LR-0.005, n_estimator = 10000, seed = i, depth = 7, submit10.csv

'15 fold CV, score: 0.8416215384358319', LR-0.008, n_estimator = 5000, seed = i, depth = 8

'20 fold CV, score: 0.8419991965085709',LR-0.008, n_estimator = 5000, seed = i, depth = 7, submit9.csv

'15 fold CV, score: 0.8419018702724905', LR-0.008, n_estimator = 5000, seed = i, depth = 7, submit8.csv, Best score on LB

'15 fold CV, score: 0.8418325697189986, LR-0.008, n_estimator = 5000, seed = 42, depth = 7, submit7.csv

'10 fold CV, score: 0.8416283637107365', LR-0.008, n_estimator = 5000, seed = 42, depth = 7, submit5.csv

'10 fold CV, score: 0.8404761359972677', LR-0.008, n_estimator = 5000, seed = 42, depth = 10, submit4.csv

'10 fold CV, score: 0.8416127641119365', LR-0.008, n_estimator = 5000, seed = 2020

'10 fold CV, score: 0.8415039416559706', LR-0.008, n_estimator = 5000, seed = 42

### Since we are using cross validation, We'll have `n` different predictions (where n is the number of fold)

In [ ]:
oof_prediction = pd.DataFrame(test_oofs).T

In [ ]:
oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [ ]:
oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10,fold_11,fold_12,fold_13,fold_14,fold_15
0,0.333959,0.302502,0.337489,0.313855,0.321676,0.287202,0.290461,0.338672,0.319495,0.304264,0.338876,0.329654,0.299721,0.314425,0.312410
1,0.369203,0.382879,0.395612,0.417410,0.383598,0.362260,0.406888,0.438215,0.357598,0.372976,0.360347,0.363000,0.400211,0.365430,0.367687
2,0.364844,0.379403,0.362060,0.380314,0.390649,0.419419,0.405681,0.380150,0.405142,0.388894,0.411739,0.381014,0.345419,0.370342,0.375410
3,0.742046,0.744472,0.738569,0.751676,0.734164,0.757131,0.751480,0.751887,0.748112,0.755135,0.773311,0.737162,0.721000,0.756357,0.736484
4,0.163131,0.146980,0.186485,0.187693,0.177555,0.165161,0.171491,0.195144,0.158370,0.170699,0.156966,0.176610,0.188350,0.171601,0.156758


In [ ]:
#sub = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/SampleSubmission.csv')

In [ ]:
### We take the mean of all 5 predictions to generate submission

In [ ]:
sub['default_status'] = np.mean(test_oofs, axis = 0)

In [ ]:
sub.to_csv('submit10.csv', index = False)

### Suggestions:
- Base your feature selection on the local cv score i.e When you add, remove or engineer feature check `score` (cell 17)
- Increase number of fold to 10, 15. See how score imporve on leaderboard.
- Try differnt value of learning rate. I'll advice anything < 0.01
- Create as many features as possible.
- Find code for Xgboost and lgbm using this method of modelling, or better still tweak this one (mind you, names of parameters may be slightly different.